# **CSC 4008 Assignment 3 (Q1.b) - 119020062 许文仲**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=49cb3cb968bda42ead1b111578e7deac94bc794c28bdeda4dbf0234a3ba854b5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenh

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/gdrive


In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [ ]:
# spark = SparkSession.builder.getOrCreate()
sc = pyspark.SparkContext.getOrCreate()
# path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/test.txt"
real_path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/graph-full.txt"
check_path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/graph-small.txt"

data = sc.textFile(real_path)
# rdd.collect()

In [ ]:
# get the first level friend RDD
size = 1000
rdd = data.map(lambda x: x.split("\t"))
rdd2 = rdd.map(lambda x: (int(x[0]), int(x[1])))
rdd_dis = rdd2.distinct()
rdd3 = rdd_dis.groupByKey().map(lambda x : (x[0], list(x[1])))
rdd4 = rdd3.sortByKey()
rdd4.collect()

[(1, [531, 689, 2, 586, 904, 502]),
 (2, [690, 632, 440, 498, 3, 505, 799, 781, 415, 713, 433]),
 (3, [545, 679, 981, 455, 619, 4, 190, 562, 796]),
 (4, [536, 986, 5, 213, 975, 953]),
 (5, [931, 453, 6, 198, 394, 704]),
 (6, [870, 206, 924, 746, 774, 662, 7, 603, 425, 909, 333]),
 (7, [453, 239, 315, 8, 6]),
 (8, [560, 128, 514, 376, 9, 947, 967, 637, 389, 151, 573]),
 (9, [347, 587, 537, 10, 964, 788, 654, 320, 226]),
 (10, [552, 11, 287, 793, 115]),
 (11, [353, 763, 163, 12, 864, 704, 898]),
 (12, [66, 734, 630, 13, 225, 263, 751, 795]),
 (13, [507, 915, 199, 701, 443, 14, 286, 358, 812, 450]),
 (14, [366, 250, 306, 718, 868, 298, 15, 761, 667, 805, 31]),
 (15, [833, 527, 737, 16, 496, 384]),
 (16, [872, 984, 692, 622, 456, 820, 414, 630, 17, 381, 195, 7]),
 (17, [593, 35, 127, 329, 995, 517, 18, 584, 340, 236, 540]),
 (18, [432, 250, 136, 19, 957]),
 (19, [291, 109, 20, 948, 286, 940, 428, 290]),
 (20, [514, 734, 762, 584, 21, 197, 159, 369]),
 (21, [467, 979, 569, 22, 942, 258, 964

In [ ]:
def test(line):
  index = line[1]
  # print(d_i)
  L_i = np.zeros(shape = (1, size))[0]
  for i in index:
    L_i[i-1] = 1
  return (np.array(L_i))

L_matrix = rdd4.map(test)
L = np.array(L_matrix.collect())
def findL(line):
  index = line[0]
  return (index, L[index-1])
L_matrix = rdd4.map(findL)
L_matrix.collect()

In [ ]:
LT = L.T
def findLT(line):
  index = line[0]
  return (index, LT[index-1])
LT_matrix = rdd4.map(findLT)
LT_matrix.collect()

In [ ]:
x = np.array(LT_matrix.collect())[0][1]
h_odd = np.full(size, 1)
np.matmul(h_odd, x)

<ipython-input-39-63b73189a57f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(LT_matrix.collect())[0][1]


11.0

In [ ]:
h_odd = np.full(size, 1)
def find_a_new(line):
  LT_i = line[1]
  a = np.matmul(h_odd, LT_i)
  return (a)
temp_a = L_matrix.map(find_a_new)
a_new = np.array(temp_a.collect())
a_new = a_new/a_new.max()

def find_h_new(line):
  L_i = line[1]
  h = np.matmul(a_new, L_i)
  return (h)
temp_h = L_matrix.map(find_h_new)
h_new = np.array(temp_h.collect())
h_new = h_new/h_new.max()

In [ ]:
iteration = 1
while iteration <= 40:
  h_odd = h_new
  temp_a = LT_matrix.map(find_a_new)
  a_new = np.array(temp_a.collect())
  a_new = a_new/a_new.max()

  temp_h = L_matrix.map(find_h_new)
  h_new = np.array(temp_h.collect())
  h_new = h_new/h_new.max()

  iteration += 1


In [ ]:
k = 5  #前k个
authorization = a_new.argsort()[-k:][::-1] + 1 #获取前k个索引
hub = h_new.argsort()[-k:][::-1] + 1
print(authorization)
print(hub)

[893  16 799 146 473]
[840 155 234 389 472]


In [ ]:
k = 5  #前k个
authorization = a_new.argsort()[:5] + 1 #获取前k个索引
hub = h_new.argsort()[:5] + 1
print(authorization)
print(hub)

[ 19 135 462  24 910]
[ 23 835 141 539 889]


In [ ]:
a_new.argsort()

array([ 18, 134, 461,  23, 909,  61, 468, 295, 423, 407, 959,  92, 628,
       943, 202, 498, 417, 317,  49, 850, 740, 822, 296, 385, 266, 429,
       443, 591, 758, 922, 925, 744, 557, 741, 109, 727, 482, 807, 378,
        37, 805, 452, 919, 141, 517, 823, 775, 177, 913, 387, 337, 799,
       656, 720, 138, 395, 718, 287, 409, 711, 706, 770, 886, 453, 953,
       619, 349, 859, 539, 133, 584, 854, 725, 500, 926, 835, 912,  54,
       936, 576, 425, 204, 686, 857, 531, 171, 633, 272, 934, 891, 954,
        33,  79, 521, 391, 503, 544, 947, 890, 653, 469, 781, 523, 641,
       270, 659, 342, 250, 315,  91,  58, 269, 815, 175, 304, 838, 321,
       644, 788, 404, 252, 639, 708, 527, 844, 555, 116, 884, 802, 128,
       247, 990, 259,  83, 410,  87, 569, 101, 487, 220,  93, 952, 445,
       853, 384, 485, 649, 826, 464, 743,  66, 492, 354, 999, 390, 438,
       657, 830, 867, 369, 618, 638,  68, 564,   4, 454, 997, 419, 673,
       111, 597, 509, 680, 397, 887, 632, 160,  80,  36, 679, 22

In [ ]:
# try to get the second level friend RDD
def findSecondLevelRdd(line):
  first_friend_list = line[1]
  for i in range(len(first_friend_list)):
    for j in range(i+1, len(first_friend_list)):
      yield((first_friend_list[i], first_friend_list[j]))
      yield((first_friend_list[j], first_friend_list[i]))

mapData = rdd4.flatMap(findSecondLevelRdd).groupByKey().map(lambda x : (x[0], list(x[1])))
mapData2 = mapData.sortByKey()
# mapData2.collect()[0]

In [ ]:

rdd10 = mapData2.union(rdd4)
rdd11 = rdd10.groupByKey().map(lambda x : (x[0], list(x[1])))
rdd12 = rdd11.sortByKey()


In [ ]:
# try to eliminate the first level friend
# 不懂为啥带不进去 带不进map里 后面解决了

from collections import *
def eliminateFirst(line):
  # second = line[1][0]
  second = line[0][1][0]
  # print(second)
  # first = line[1][1]
  first = line[0][1][1]
  # print(first)
  rdd_data_second = sc.parallelize(second)
  rdd_data_first = sc.parallelize(first)
  result = rdd_data_second.subtract(rdd_data_first).collect()
  result = sorted(result)
  # print(result)
  mutual = Counter(result).most_common(10)
  return (line[0][0], mutual)

# eliminateFirst(rdd12.collect()[0])

# getFriends = rdd12.map(eliminateFirst)
# getFriends.collect()

In [ ]:
# this is for whom only wants for a single index
def getResult(index):
  wanted_RDD = rdd12.filter(lambda x: x[0] == index)
# wanted_RDD.collect()[0]
  print(eliminateFirst(wanted_RDD.collect()))

wanted = [1, 10, 152, 288, 603, 714, 1525, 2434, 2681]
for i in wanted:
  getResult(i)


(1, [(710, 4), (603, 3), (714, 2), (1525, 2), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1)])
(10, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (12, 1)])
(152, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)])
(288, [(71, 3), (1525, 3), (69, 2), (90, 2), (217, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2), (2356, 2)])
(603, [(1, 3), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1), (295, 1), (296, 1), (297, 1)])
(714, [(1, 2), (712, 2), (713, 2), (715, 2), (717, 2), (718, 2), (1525, 2), (90, 1), (217, 1), (247, 1)])
(1525, [(288, 3), (1, 2), (710, 2), (714, 2), (603, 1)])
(2434, [(71, 2), (288, 2), (711, 2), (716, 2), (719, 2), (720, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2)])
(2681, [(71, 1), (288, 1), (710, 1), (711, 1), (716, 1), (719, 1), (720, 1), (721, 1), (722, 1), (2348, 1)])


In [ ]:
# useless
# rdd_union.groupByKey().collect()
# index = 1571
# type(rdd_union.lookup(index))
# first_level_friend_list = sorted(rdd_union.lookup(index))
# second_level_friend_list = []
# for first_level_friend in first_level_friend_list:
#   second_level_friend_list += rdd_union.lookup(first_level_friend)


In [ ]:
# 需要在 二级朋友 中 除去自身 和 一级朋友 (useless)
# second_level_friend_list = sorted(second_level_friend_list)
# # second_level_friend_set = set(second_level_friend_list)
# first_level_friend_set = set(first_level_friend_list)
# first_level_friend_set.add(index)
# # diff = second_level_friend_set - first_level_friend_set
# # diff_list = list(diff)
# # diff_list.remove(1571)
# for i in first_level_friend_set:
#   while i in second_level_friend_list:
#     second_level_friend_list.remove(i)


In [ ]:
# useless
# from collections import *
# result = Counter(second_level_friend_list).most_common(10)
# if result == []:
#   output = []
# else:
#   output = [i[0] for i in result]
# output

[710, 603, 714, 1525, 289, 290, 291, 292, 293, 294]

In [ ]:
# compute the total result by 'map' so that you can pick the one you want
from collections import *
def eliminateFirst(line):
  # second = line[1][0]
  second_level_friend_list = line[1][0]
  # print(second)
  # first = line[1][1]
  first_level_friend_list = line[1][1]
  # print(first)
  # rdd_data_second = sc.parallelize(second)
  # rdd_data_first = sc.parallelize(first)
  # result = rdd_data_second.subtract(rdd_data_first).collect()
  # second_level_friend_list = sorted(second_level_friend_list)
  # second_level_friend_set = set(second_level_friend_list)
  first_level_friend_set = set(first_level_friend_list)
  # diff = second_level_friend_set - first_level_friend_set
  # diff_list = list(diff)
  # diff_list.remove(1571)
  for i in first_level_friend_set:
    while i in second_level_friend_list:
      second_level_friend_list.remove(i)
  second_level_friend_list = sorted(second_level_friend_list)
  result = Counter(second_level_friend_list).most_common(10)

  if result == []:
    output = []
  else:
    output = result
    wrong = [i[0] for i in result]
    # print(wrong)
  # return (line[0], output)
  return (line[0], wrong)

getFriends = rdd12.map(eliminateFirst)
# getFriends.collect()

In [ ]:
# getFriends.collect()
wanted = [10, 152, 288, 603, 714, 1525, 2434, 2681]
result = getFriends.filter(lambda x: x[0] in wanted).collect()
for each in sc.parallelize(result).sortBy(lambda pair: pair[0]).collect():
    print(each)

(10, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (12, 1)])
(152, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)])
(288, [(71, 3), (1525, 3), (69, 2), (90, 2), (217, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2), (2356, 2)])
(603, [(1, 3), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1), (295, 1), (296, 1), (297, 1)])
(714, [(1, 2), (712, 2), (713, 2), (715, 2), (717, 2), (718, 2), (1525, 2), (90, 1), (217, 1), (247, 1)])
(1525, [(288, 3), (1, 2), (710, 2), (714, 2), (603, 1)])
(2434, [(71, 2), (288, 2), (711, 2), (716, 2), (719, 2), (720, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2)])
(2681, [(71, 1), (288, 1), (710, 1), (711, 1), (716, 1), (719, 1), (720, 1), (721, 1), (722, 1), (2348, 1)])


In [ ]:
wanted = [10, 152, 288, 603, 714, 1525, 2434, 2681]
result = getFriends.filter(lambda x: x[0] in wanted).collect()
for each in sc.parallelize(result).sortBy(lambda pair: pair[0]).collect():
    print(each)

(10, [2, 3, 4, 5, 6, 7, 8, 9, 11, 12])
(152, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
(288, [71, 1525, 69, 90, 217, 2348, 2351, 2352, 2354, 2356])
(603, [1, 289, 290, 291, 292, 293, 294, 295, 296, 297])
(714, [1, 712, 713, 715, 717, 718, 1525, 90, 217, 247])
(1525, [288, 1, 710, 714, 603])
(2434, [71, 288, 711, 716, 719, 720, 2348, 2351, 2352, 2354])
(2681, [71, 288, 710, 711, 716, 719, 720, 721, 722, 2348])
